In [ ]:
import torch

m, n, p = 100, 4, 10

def grads(x):
    m, n, p = x.shape
    n_flat = n * p
    
    x = x.reshape(m, n_flat)
    x_flat = [x[..., i].requires_grad_(True) for i in range(n_flat)]
    x_prime = torch.stack(x_flat, -1)
    x = x_prime.view(m, n, p)
    y = function(x)
    
    dy_dx = torch.autograd.grad(y.sum(), x_prime, create_graph=True, retain_graph=True)[0]
    dy_dx_prime = torch.split(dy_dx, 1, -1)
    
    laplace = [torch.autograd.grad(g.sum(), xt, retain_graph=True)[0] for g, xt in zip(dy_dx_prime, x_flat)]
    
    return torch.stack(laplace)

def function(x):
    w = torch.normal(0., 1., (x.shape[-1],1), device=x.device)
    return (x @ w).sum(1).tanh()
    
x = torch.normal(0., 1., (m, n, p)).to('cuda')

grads(x)

# def laplacian(model: nn.Module, walkers: tc.Tensor) -> Tuple[tc.Tensor, tc.Tensor]:
#     n_walkers, n_el = walkers.shape[:2]
#     n_r = 3 * n_el

#     walkers = walkers.reshape(n_walkers, n_r)
#     r_s = [walkers[..., i].requires_grad_(True) for i in range(n_r)]
#     positions = tc.stack(r_s, -1)
#     walkers = positions.view(n_walkers, n_el, 3)
#     log_phi = model(walkers)

#     dlogphi_dr = tc.autograd.grad(log_phi.sum(), positions, create_graph=True, retain_graph=True)[0]
#     grads = tc.split(dlogphi_dr, 1, -1)

#     d2logphi_dr2 = tc.stack([tc.autograd.grad(grad.sum(), r, retain_graph=True)[0] for grad, r in zip(grads, r_s)], 1)

#     return dlogphi_dr**2, d2logphi_dr2